<a href="https://colab.research.google.com/github/mdehghani86/AppliedGenAI/blob/main/LangChain_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 26px 28px 18px 28px; border-radius: 14px; margin-bottom: 22px; font-family: Arial, sans-serif;">
  <h2 style="margin-top: 0; font-size: 27px; letter-spacing: 0.5px;">
    🧠 LangChain Lab 2: Prompt Templates &amp; Memory
  </h2>
  <p style="font-size: 17px; margin-bottom: 8px;">
    <span style="color: #a5d8ff;">Instructor:</span> Prof. Dehghani
  </p>
  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 12px;">Lab Overview</h3>
  <p style="font-size: 16px; margin-bottom: 14px;">
    In this lab, you'll enhance your interactions with LLMs by using <b>Prompt Templates</b> and <b>Memory</b> features in LangChain.<br>
    You'll learn to create <b>structured prompts dynamically</b> and maintain <b>conversation history</b> across multiple turns.
  </p>
  <hr style="border: 1px solid #3f77d4; margin: 16px 0;">
  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 10px;">🎯 What You'll Learn</h3>
  <ul style="font-size: 16px; margin: 0 0 10px 22px; line-height: 1.8;">
    <li>🔹 <b>Prompt Templates</b> – Format inputs dynamically for LLMs.</li>
    <li>🔹 <b>Memory in LangChain</b> – Maintain context in multi-turn conversations.</li>
    <li>🔹 <b>Hands-on exercises</b> – Reinforce concepts with practical coding tasks.</li>
  </ul>
  <p style="font-size: 15.5px; margin-top: 8px;">
    By the end, you'll be able to structure prompts effectively and implement conversational memory in LangChain applications. 🚀
  </p>
</div>


##⚙️ Install essential packages

In [ ]:
#⚙️ Install essential packages for LangChain with OpenAI & Gemini support

!pip install -q --upgrade langchain                # Core LangChain framework for building LLM workflows
!pip install -q --upgrade langchain-community      # Community integrations (still useful for many non-OpenAI/Gemini models)
!pip install -q --upgrade langchain-openai         # ✅ NEW: Dedicated package for OpenAI integrations
!pip install -q --upgrade langchain-google-genai   # Integration for Google's Gemini models
!pip install -q --upgrade openai                   # OpenAI SDK for native API calls (not strictly needed for LangChain, but often useful)

##🔑 Step 2: Set Up OpenAI API Key

In [ ]:
# ⚙️ Load API Keys from Colab Secrets
# ==================================

import os                                  # Used to set environment variables for API keys
from google.colab import userdata          # To securely access stored secrets in Colab

# Retrieve your stored secrets (API keys)
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')   # OpenAI API key for GPT models
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')   # Google Gemini API key for Gemini models

# Set environment variables for the APIs and confirm success
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY   # Set OpenAI key as environment variable
    print("✅ OpenAI API key loaded successfully!")
else:
    print("❌ OpenAI API key not found. Please set 'OPENAI_API_KEY' in Colab secrets.")

if GEMINI_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY   # Set Gemini key as environment variable
    print("✅ Google Gemini API key loaded successfully!")
else:
    print("❌ Google Gemini API key not found. Please set 'GEMINI_API_KEY' in Colab secrets.")

<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 26px 28px 18px 28px; border-radius: 14px; margin-bottom: 22px; font-family: Arial, sans-serif;">
  <h2 style="margin-top: 0; font-size: 25px; letter-spacing: 0.5px;">📝 Prompt Templates in LangChain</h2>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 8px;">🔹 What are Prompt Templates?</h3>
  <p style="font-size: 16px; margin-bottom: 12px;">
    Prompt Templates let you <b>dynamically format prompts</b> by inserting variables, making interactions with LLMs more flexible and reusable.<br>
    Instead of writing static text, you can use placeholders that are filled in with real values at runtime.
  </p>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 8px;">🔹 Why Use Prompt Templates?</h3>
  <ul style="font-size: 16px; margin: 0 0 14px 22px; line-height: 1.8;">
    <li>✅ <b>Reusability</b> – No need for repetitive prompts.</li>
    <li>✅ <b>Dynamic Inputs</b> – Easily personalize prompts with new user data.</li>
    <li>✅ <b>Consistency</b> – Keeps your prompt formatting structured and reliable.</li>
  </ul>

  <h3 style="font-size: 17px; margin-bottom: 8px;">📌 Example Usage</h3>
  <div style="background: rgba(255,255,255,0.08); border-radius: 7px; padding: 11px 16px; margin-bottom: 6px;">
    <span style="color: #a5d8ff;">Static prompt:</span><br>
    <span style="font-family: 'Fira Mono', monospace; font-size: 15px; color: #fff;">
      "What are the benefits of AI in healthcare?"
    </span>
  </div>
  <div style="background: rgba(255,255,255,0.08); border-radius: 7px; padding: 11px 16px;">
    <span style="color: #a5d8ff;">Dynamic prompt with a template:</span><br>
    <span style="font-family: 'Fira Mono', monospace; font-size: 15px; color: #fff;">
      "What are the benefits of {technology} in {industry}?"
    </span><br>
    <span style="font-size: 15px;">If <b>{technology} = "AI"</b> and <b>{industry} = "healthcare"</b>, the prompt becomes:</span><br>
    <span style="font-family: 'Fira Mono', monospace; font-size: 15px; color: #fff;">
      "What are the benefits of AI in healthcare?"
    </span>
  </div>

  <p style="font-size: 16px; margin-top: 14px;">🚀 Let's get started with the first example!</p>
</div>


In [ ]:
"""
# ==================================================
# 🎯 Using Prompt Templates with OpenAI (GPT-4)
# ==================================================
"""

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI   # ✅ NEW import for ChatOpenAI

# Step 1: Define a prompt template with variables
prompt_template = PromptTemplate(
    input_variables=["technology", "industry"],
    template="What are the benefits of {technology} in {industry} in 1 sentence?"
)

# Step 2: Format the prompt with specific values
formatted_prompt = prompt_template.format(technology="AI", industry="healthcare")

# Step 3: Initialize the OpenAI LLM (GPT-4)
llm_ChatGPT = ChatOpenAI(model_name="gpt-4")

# Step 4: Generate the response
response_ChatGPT = llm_ChatGPT.invoke(formatted_prompt)

# Step 5: Display results
print("🔹 Generated Prompt:", formatted_prompt)
print("🔹 LLM Response:", response_ChatGPT.content)


In [ ]:
# ==================================================
# ✋ **Hands-On: Creating Dynamic Prompt Templates**
# ==================================================

# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (-----) to complete the code.
# 2️⃣ Ensure the Prompt Template correctly replaces {topic} and {context}.
# 3️⃣ Run the code and verify GPT-4 generates a response.

# ✅ Step 1: Define a Prompt Template
prompt_template = -----(
    input_variables=["-----", "context"],  # Fill in the missing variable name
    template="How does {topic} impact {context} in a few words?"  # Structure of the prompt
)

# ✅ Step 2: Format the prompt with actual values
formatted_prompt = prompt_template.----- (topic="Machine Learning", -----="business analytics")

# ✅ Step 3: Generate a response using OpenAI (GPT-4)
llm_ChatGPT = llm.----- (model_name="gpt-4")  # Initialize the ChatGPT model
response_ChatGPT = llm_ChatGPT.invoke(-----)  # Fill in the correct variable for invoke

# ✅ Step 4: Display results
print("🔹 **Generated Prompt:**", formatted_prompt)
print("🔹 **LLM Response:**", response_ChatGPT.-----)  # Extract response content


In [ ]:
"""
# ==================================================
# 🔄 Using Prompt Templates in a Loop
# ==================================================
"""

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # ✅ NEW import—no deprecation warning!

# Step 1️⃣: Define a prompt template with variables
prompt_template = PromptTemplate(
    input_variables=["technology", "industry"],
    template="In one sentence, how does {technology} impact {industry}?"
)

# Step 2️⃣: Initialize the OpenAI LLM (GPT-4)
llm_ChatGPT = ChatOpenAI(model_name="gpt-4")

# Step 3️⃣: Define input values for the loop
input_data = [
    {"technology": "AI", "industry": "education"},
    {"technology": "Blockchain", "industry": "finance"},
    {"technology": "5G", "industry": "telecommunications"},
]

# Step 4️⃣: Loop through inputs, format the prompt, and generate a response
for data in input_data:
    formatted_prompt = prompt_template.format(**data)
    response_ChatGPT = llm_ChatGPT.invoke(formatted_prompt)

    # Step 5️⃣: Display results in a clear, modern format
    print(f"🔹 Prompt: {formatted_prompt}")
    print(f"💡 Response: {response_ChatGPT.content}")
    print("-" * 60)

# 🧠 **Understanding Memory in LangChain**

## 🔹 **What is Memory in LangChain?**
By default, LLMs **do not remember past interactions**.  
LangChain **Memory** allows an AI model to **retain context** across multiple turns, enabling more natural, conversational interactions.  

## 🔹 **Why Use Memory?**
✅ **Maintains conversation history** → AI can recall previous exchanges.  
✅ **Improves response coherence** → Reduces redundant user re-explanations.  
✅ **Essential for chatbots & agents** → Allows multi-turn dialogue without loss of context.  

## 🔹 **Types of Memory in LangChain**
LangChain provides **various types of memory**, including:  
1️⃣ **ConversationBufferMemory** → Stores messages in a buffer (basic memory).  
2️⃣ **ConversationSummaryMemory** → Summarizes past interactions instead of storing all messages.  
3️⃣ **ConversationBufferWindowMemory** → Retains only the last N interactions for efficiency.  
4️⃣ **Vector-based Memory** → Uses embeddings for advanced retrieval of past conversations.  

## **🚀 What We'll Do in This Lab**
We’ll explore **ConversationBufferMemory** first, which allows an LLM to **recall past messages** and interact in a more natural, memory-enhanced way.  

Let's get started! 👇  


In [ ]:
# ==================================================
# 🔢 **Using Memory in LangChain: Math Example**
# ==================================================
#
# This script demonstrates how to use memory in LangChain for multi-turn conversations.
# It initializes a conversation with memory and interacts with GPT-4 to show that it
# can recall previous user inputs and continue reasoning based on past responses.


# ✅ Import required classes
from langchain.memory import ConversationBufferMemory  # Memory system for storing conversation history
from langchain.chains import ConversationChain  # Handles multi-turn conversations
from langchain.chat_models import ChatOpenAI  # Import ChatOpenAI to use GPT models

# ✅ Step 1: Initialize Memory
memory = ConversationBufferMemory()  # Stores conversation history

# ✅ Step 2: Initialize ChatGPT with Memory
llm = ChatOpenAI(model_name="gpt-4")  # Using GPT-4 model

# ✅ Step 3: Initialize Conversation Chain
# This step is crucial as it links the language model with memory, enabling it to retain
# and recall past interactions, creating a coherent multi-turn conversation.
conversation = ConversationChain(llm=llm, memory=memory)  # Attaching memory

# ✅ Step 4: Run Multiple Interactions
# The `conversation.predict()` method is crucial because it retrieves responses
# while maintaining context from previous exchanges stored in memory.
# This enables the model to remember past interactions and generate relevant answers.

print("\n💬 **User:** What is 15 + 27?")
response = conversation.predict(input="What is 15 + 27?")
print("🤖 **ChatGPT:**", response)

print("\n💬 **User:** Now multiply that result by 3.")
response = conversation.predict(input="Now multiply that result by 3.")
print("🤖 **ChatGPT:**", response)

print("\n💬 **User:** What was my first question?")
response = conversation.predict(input="What was my first question?")
print("🤖 **ChatGPT:**", response)


## 🔢 **Using Memory in LangChain: Math Example (Hugging Face)**


In [ ]:
# ==================================================
# 🔢 **Using Memory in LangChain: Math Example (Hugging Face)**
# ==================================================

# ✅ Import required libraries
import os
from langchain.memory import ConversationBufferMemory  # Stores conversation history
from langchain.chains import ConversationChain  # Handles multi-turn conversations
from langchain.chat_models import ChatOpenAI  # Chat model integration
from langchain.llms import HuggingFaceHub  # Hugging Face Model integration

# ✅ Step 1: Set Hugging Face API Token (Ensure this is set securely beforehand)
if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
    raise ValueError("Please set your Hugging Face API token as an environment variable.")

# ✅ Step 2: Initialize the Hugging Face model
llm_HF = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature": 0.5})  # Falcon model

# ✅ Step 3: Initialize Memory
memory = ConversationBufferMemory()  # Stores conversation history

# ✅ Step 4: Attach memory to a conversation chain
conversation = ConversationChain(llm=llm_HF, memory=memory)  # Attaching memory

# ✅ Step 5: Run Multiple Interactions
print("\n💬 **User:** What is 12 + 8?")
response_HF = conversation.predict(input="What is 12 + 8?")
print("🤖 **Falcon Model:**", response_HF)

print("\n💬 **User:** Now subtract 5 from that result.")
response_HF = conversation.predict(input="Now subtract 5 from that result.")
print("🤖 **Falcon Model:**", response_HF)

print("\n💬 **User:** What was my first question?")
response_HF = conversation.predict(input="What was my first question?")
print("🤖 **Falcon Model:**", response_HF)


In [ ]:
# ==================================================
# 🔎 **Search for Hugging Face Models**
# ==================================================

import requests  # To send API requests
import json  # For handling responses

# ✅ Step 1: Define the Hugging Face API URL
HF_API_URL = "https://huggingface.co/api/models"

# ✅ Step 2: Define search parameters
query = "chat"  # Change this to search for different models (e.g., "math", "finance", "healthcare")
params = {"search": query, "limit": 10}  # Adjust limit as needed

# ✅ Step 3: Send a request to Hugging Face Model Hub
response = requests.get(HF_API_URL, params=params)

# ✅ Step 4: Display results
if response.status_code == 200:
    models = response.json()
    print(f"🔍 **Top {len(models)} Models Matching '{query}':**")
    for model in models:
        print(f"🔹 {model['modelId']} ➝ {model['pipeline_tag']}")  # Show model ID and type
else:
    print("❌ Failed to retrieve models. Check your internet connection.")


✋ Hands-On 1: Memory with ChatGPT (OpenAI)

In [ ]:
# ==================================================
# ✋ **Hands-On: Using Memory with OpenAI (Beer Game - Supply Chain Predictions)**
# ==================================================
#
# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (-----) to complete the code.
# 2️⃣ Ensure the AI remembers previous demand data and predicts future order quantities.
# 3️⃣ Run the code and check if ChatGPT maintains context for supply chain decisions.

# ✅ Step 1: Initialize Memory
memory = -----()  # Initialize the correct memory class

# ✅ Step 2: Initialize ChatGPT with Memory
llm_ChatGPT = ----- (model_name="gpt-4")  # Initialize ChatGPT model
conversation = ----- (llm=llm_ChatGPT, memory=memory)  # Attach memory to conversation

# ✅ Step 3: Run Multiple Interactions
print("\n💬 **Retailer:** Last week, the customer demand was 200 units. What should I order this week?")
response_ChatGPT = conversation.----- (input="Last week, the customer demand was 200 units. What should I order this week?")  # Call the correct method
print("🤖 **ChatGPT:**", response_ChatGPT)

print("\n💬 **Retailer:** If demand increases by 10%, how many units should I prepare for next week?")
response_ChatGPT = conversation.----- (input="If demand increases by 10%, how many units should I prepare for next week?")
print("🤖 **ChatGPT:**", response_ChatGPT)

print("\n💬 **Retailer:** What was the demand I mentioned last week?")
response_ChatGPT = conversation.----- (input="What was the demand I mentioned last week?")
print("🤖 **ChatGPT:**", response_ChatGPT)


## Using 'Summarized Conversation' Example


In [ ]:
# ==================================================
# 🎤 **Using Memory in LangChain: Job Interview Prep**
# ==================================================
#
# This script simulates a job interview practice session.
# It uses ConversationSummaryMemory to retain key points from previous exchanges
# rather than storing the full conversation history.

# ✅ Import required classes
from langchain.memory import ConversationSummaryMemory  # Summarized conversation memory

# ✅ Step 1: Initialize Memory
# This memory will maintain a **summarized** version of the conversation.
memory = ConversationSummaryMemory(llm=ChatOpenAI(model_name="gpt-4"))

# ✅ Step 2: Initialize ChatGPT with Memory
llm = ChatOpenAI(model_name="gpt-4")  # Using GPT-4 model

# ✅ Step 3: Initialize Conversation Chain
# The model will summarize key details from the job interview practice.
conversation = ConversationChain(llm=llm, memory=memory)

# ✅ Step 4: Conduct the Interview Simulation

print("\n💬 **User:** Can you ask me a common interview question?")
response = conversation.predict(input="Can you ask me a common interview question?")
print("🤖 **ChatGPT:**", response)

# ✅ Check memory after first interaction
print("\n📜 **Memory Summary After 1st Question:**")
print(memory.load_memory_variables({})["history"])

print("\n💬 **User:** My biggest strength is adaptability and problem-solving.")
response = conversation.predict(input="My biggest strength is adaptability and problem-solving.")
print("🤖 **ChatGPT:**", response)

# ✅ Check memory after user shares strength
print("\n📜 **Memory Summary After Strength Response:**")
print(memory.load_memory_variables({})["history"])

print("\n💬 **User:** My biggest weakness is that I sometimes overthink decisions.")
response = conversation.predict(input="My biggest weakness is that I sometimes overthink decisions.")
print("🤖 **ChatGPT:**", response)

# ✅ Check memory after user shares weakness
print("\n📜 **Memory Summary After Weakness Response:**")
print(memory.load_memory_variables({})["history"])

print("\n💬 **User:** Can you summarize what we discussed so far?")
response = conversation.predict(input="Can you summarize what we discussed so far?")
print("🤖 **ChatGPT:**", response)

# ✅ Final Memory Check
print("\n📜 **Final Memory Summary:**")
print(memory.load_memory_variables({})["history"])


In [ ]:
# ==================================================
# 🍺 **LangChain Beer Game: Comparing Memory Types (Fixed)**
# ==================================================
#
# This script simulates a Beer Game ordering process over 6 weeks.
# It uses:
# 1️⃣ ConversationBufferMemory (Tracks full history)
# 2️⃣ ConversationBufferWindowMemory (Tracks only last 3 orders)
#
# The AI predicts the next order quantity based on past interactions.

# ✅ Import required libraries
import pandas as pd
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# ✅ Step 1: Initialize Memory Types
buffer_memory = ConversationBufferMemory(return_messages=True)  # Stores entire history
window_memory = ConversationBufferWindowMemory(k=3, return_messages=True)  # Stores last 3 interactions

# ✅ Step 2: Initialize Chat Model
llm = ChatOpenAI(model_name="gpt-4")

# ✅ Step 3: Define a Prompt Template
beer_game_template = PromptTemplate(
    input_variables=["context"],
    template="""
    You are managing a supply chain for a beer distribution system.
    Orders fluctuate at first but stabilize later.

    {context}

    Based on past orders, what should be the next order quantity?
    """
)

# ✅ Step 4: Define Processing Pipelines
buffer_chain = beer_game_template | llm
window_chain = beer_game_template | llm

# ✅ Step 5: Define Order Fluctuations (First 3 weeks volatile, last 3 weeks stable)
weekly_orders = [20, 50, 10, 25, 30, 30]  # Example fluctuations

# Store results for comparison
buffer_memory_log = []
window_memory_log = []
buffer_predictions = []
window_predictions = []

# ✅ Step 6: Run the Simulation
for week in range(1, len(weekly_orders) + 1):
    prev_orders = ", ".join(map(str, weekly_orders[:week]))  # Orders seen so far
    context = f"Week {week}: The previous orders were {prev_orders}."

    # Store input in memory
    buffer_memory.save_context({"context": context}, {"response": ""})
    window_memory.save_context({"context": context}, {"response": ""})

    # Get AI predictions using RunnableSequence
    buffer_prediction = buffer_chain.invoke({"context": context})
    window_prediction = window_chain.invoke({"context": context})

    # Retrieve memory states
    buffer_memory_summary = buffer_memory.load_memory_variables({})["history"]
    window_memory_summary = window_memory.load_memory_variables({})["history"]

    # Store memory states and predictions
    buffer_memory_log.append(buffer_memory_summary)
    window_memory_log.append(window_memory_summary)
    buffer_predictions.append(buffer_prediction.content)
    window_predictions.append(window_prediction.content)

# ✅ Step 7: Display Results in a Table
df = pd.DataFrame({
    "Week": list(range(1, len(weekly_orders) + 1)),
    "Actual Orders": weekly_orders,
    "Buffer Memory (Stores All)": buffer_memory_log,
    "Window Memory (Last 3 Turns)": window_memory_log,
    "Buffer Memory Prediction": buffer_predictions,
    "Window Memory Prediction": window_predictions
})


In [ ]:
# ✅ Save the table to an Excel file
df.to_excel("beer_game_memory_comparison.xlsx", index=False)

# ✅ Print confirmation message
print("Data saved to 'beer_game_memory_comparison.xlsx'")


# 📌 **Assignment: AI Stock Market Trend Prediction with Memory**

## **Objective**
In this assignment, you will use AI to predict stock market trends based on historical stock prices. You will compare how different memory types affect AI's ability to track and predict future trends.

## **Tasks**
1. **Initialize memory types** (`ConversationBufferMemory` and `ConversationBufferWindowMemory`).
2. **Define the AI model** (GPT-4 or another suitable model).
3. **Complete the prompt template** to guide AI predictions.
4. **Process stock price data** and use memory to store past trends.
5. **Retrieve and analyze stored memory** after each step.
6. **Invoke the AI model correctly** to generate predictions.
7. **Save results to an Excel file** for analysis.

## **Expected Outcome**
You will observe how AI predictions change when it has full history vs. limited memory. This will help you understand the impact of memory in AI-based forecasting.

🚀 **Complete the placeholders and run the script to generate insights!** 🚀


In [ ]:
# ==================================================
# 📈 **AI Assignment: Stock Market Trend Prediction with Memory**
# ==================================================
#
# 🔹 In this assignment, you will use AI to predict stock market trends.
# 🔹 You will compare how different memory types affect AI's ability to track stock price movements.
# 🔹 Complete the placeholders (----) to make the script functional.
#
# 📌 **Your Tasks:**
# 1️⃣ Initialize the correct memory types.
# 2️⃣ Define the AI model.
# 3️⃣ Complete the template prompt.
# 4️⃣ Use memory correctly when processing stock data.
# 5️⃣ Ensure correct invocation of AI for predictions.
# 6️⃣ Retrieve and analyze stored memory.
# 7️⃣ Save results in an Excel file.

# ✅ Import required libraries
import pandas as pd
from langchain.memory import ----  # Import appropriate memory classes
from langchain.chat_models import ----  # Import ChatGPT model
from langchain.prompts import ----  # Import PromptTemplate

# ✅ Step 1: Initialize Memory Types
buffer_memory = ----  # Stores full stock history
window_memory = ----  # Stores last 3 stock movements

# ✅ Step 2: Initialize Chat Model
llm = ----  # Define the AI model (GPT-4 or another model)

# ✅ Step 3: Define a Prompt Template
stock_prediction_template = PromptTemplate(
    input_variables=["context"],
    template="""
    You are an AI financial analyst predicting stock market trends.

    {context}

    Based on this stock price history, what will be the next trend (Up, Down, or Stable)?
    """
)

# ✅ Step 4: Define Processing Pipelines
buffer_chain = ----  # Define how memory connects to AI
window_chain = ----  # Define how memory connects to AI with windowed memory

# ✅ Step 5: Define Stock Price Data (Fluctuations in the first weeks, then stabilizing)
stock_prices = [120, 125, 110, 130, 128, 129]  # Example price movements

# Store results for comparison
buffer_memory_log = []
window_memory_log = []
buffer_predictions = []
window_predictions = []

# ✅ Step 6: Run the Prediction Simulation
for week in range(1, len(stock_prices) + 1):
    prev_prices = ", ".join(map(str, stock_prices[:week]))  # Stocks seen so far
    context = f"Week {week}: The previous stock prices were {prev_prices}."

    # Store input in memory
    buffer_memory.----  # Store context in buffer memory
    window_memory.----  # Store context in windowed memory

    # Get AI predictions
    buffer_prediction = buffer_chain.----  # Invoke AI for buffer memory
    window_prediction = window_chain.----  # Invoke AI for window memory

    # Retrieve memory states
    buffer_memory_summary = buffer_memory.load_memory_variables({})["history"]
    window_memory_summary = window_memory.load_memory_variables({})["history"]

    # Store memory states and predictions
    buffer_memory_log.append(buffer_memory_summary)
    window_memory_log.append(window_memory_summary)
    buffer_predictions.append(buffer_prediction.content)
    window_predictions.append(window_prediction.content)

# ✅ Step 7: Save Results in an Excel File
df = pd.DataFrame({
    "Week": list(range(1, len(stock_prices) + 1)),
    "Stock Price": stock_prices,
    "Buffer Memory (Stores All)": buffer_memory_log,
    "Window Memory (Last 3 Turns)": window_memory_log,
    "Buffer Memory Prediction": buffer_predictions,
    "Window Memory Prediction": window_predictions
})

df.to_excel("stock_market_memory_comparison.xlsx", index=False)

# ✅ Print confirmation message
print("Assignment completed! Data saved to 'stock_market_memory_comparison.xlsx'")
